In [46]:
!pip install pandas-plink limix-lmm


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [47]:
import numpy as np
import pdb
from pandas_plink import read_plink
from limix_lmm import LMM
import pylab as plt
import os
import pandas as pd

In [48]:
import scipy
import scipy.stats as st

if not hasattr(scipy, 'dot'):
    scipy.dot = np.dot
if not hasattr(scipy, 'einsum'):
    scipy.einsum = np.einsum
if not hasattr(scipy, 'log'):
    scipy.log = np.log
if not hasattr(scipy, 'sign'):
    scipy.sign = np.sign
if not hasattr(scipy, 'sqrt'):
    scipy.sqrt = np.sqrt

In [49]:
!wget -O ALL.chr22_GRCh38.genotypes.20170504.zip "https://www.dropbox.com/s/86fdj3i7ir3vap7/ALL.chr22_GRCh38.genotypes.20170504.zip?dl=1"
!unzip -o -q ALL.chr22_GRCh38.genotypes.20170504.zip && rm ALL.chr22_GRCh38.genotypes.20170504.zip

--2026-02-12 13:16:54--  https://www.dropbox.com/s/86fdj3i7ir3vap7/ALL.chr22_GRCh38.genotypes.20170504.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6022:18::a27d:4212, 162.125.66.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6022:18::a27d:4212|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/2j079oew31gwd2hzajlil/ALL.chr22_GRCh38.genotypes.20170504.zip?rlkey=7307x4sjq9lkvhwu3aivh5uut&dl=1 [following]
--2026-02-12 13:16:55--  https://www.dropbox.com/scl/fi/2j079oew31gwd2hzajlil/ALL.chr22_GRCh38.genotypes.20170504.zip?rlkey=7307x4sjq9lkvhwu3aivh5uut&dl=1
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc151c5ed2aa0b32dc07b0e7c465.dl.dropboxusercontent.com/cd/0/inline/C6yag50KEyC8XfwRRYqXEMlRjaothflWjq3WSNnmnzgGLFX33OvZFik4LXlny0gobsygjRfVWla7kq1KC1s6RAUphNjxJzmFLgTFpM7eR0522ITvMP9sMwifLFs2azi2q9A/file?dl=1# [following]
--

In [50]:
# load genetic data
bfile = 'ALL.chr22_GRCh38.genotypes.20170504'
bim, fam, G = read_plink(bfile)

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  9.69it/s]


Preprocessing

In [51]:
print(G.shape)   # (n_variants, n_samples)
print(bim.head())
print(fam.head())
print(G.shape[0], bim.shape[0], fam.shape[0])


(109827, 2504)
  chrom          snp   cm       pos a0 a1  i
0    22    rs9617549  0.0  10874444  T  C  0
1    22    rs5747224  0.0  11122151  A  G  1
2    22    rs2456393  0.0  11122417  T  G  2
3    22  rs199668908  0.0  11123683  C  T  3
4    22    rs4010319  0.0  11123736  C  T  4
       fid      iid father mother gender trait  i
0  HG00096  HG00096      0      0      0    -9  0
1  HG00097  HG00097      0      0      0    -9  1
2  HG00099  HG00099      0      0      0    -9  2
3  HG00100  HG00100      0      0      0    -9  3
4  HG00101  HG00101      0      0      0    -9  4
109827 109827 2504


In [52]:
#remove variants with many missing data
snp_missing = np.mean(np.isnan(G), axis=1).compute()
keep_missing = snp_missing < 0.01

G = G[keep_missing, :]
bim = bim.iloc[keep_missing].reset_index(drop=True)

In [53]:
#removing duplicates of variants to avoid bias
duplicate = bim.duplicated(subset=['chrom', 'pos'], keep = 'first')

G.compute_chunk_sizes()
G = G[~duplicate, :]
bim = bim.loc[~duplicate].reset_index(drop=True)


In [54]:
#retaining only common variants -MAF >=1% or 5%
allele_freq = np.nanmean(G, axis=1)/2 
maf = np.minimum(allele_freq, 1-allele_freq)

keep_maf = maf > 0.01
G = G[keep_maf, :]
bim = bim.iloc[keep_maf].reset_index(drop=True)
G.compute_chunk_sizes()  

dask.array<getitem_variadic, shape=(109712, 2504), dtype=float32, chunksize=(1016, 1024), chunktype=numpy.ndarray>

In [55]:
def extract_region(bim, G, chrom, start, end):
    I1 = bim['chrom']==str(chrom)
    I2 = bim['pos'] > start
    I3 = bim['pos'] < end
    Ikeep = I1 * I2 * I3
    bim = bim.loc[Ikeep].copy()
    G = G[bim['i'].values]
    bim['i'] = np.arange(len(bim))
    return bim, G

In [56]:
def subset_individuals(fam, G, keep_idxs):
    fam = fam.iloc[keep_idxs]
    G = G[:, keep_idxs]
    return bim, G

In [57]:
# subset to 1000 individuals
N = 1000
np.random.seed(0)
idxs = np.sort(np.random.permutation(fam.shape[0])[:N])
fams, Gs = subset_individuals(fam, G, idxs)

In [58]:
# extract region
chrom = 22
start = 30797531
size = 100000
bimr, Gr = extract_region(bim, Gs, chrom, start, start + size)

In [59]:
# load genotype matrix
X_real = Gr.compute().T

Phase 1: Phenotype Simulation